<a href="https://colab.research.google.com/github/acsontfd/NewsScrapers/blob/main/assessment_CGT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [198]:
%pip install google-colab-selenium

In [199]:
%pip install google-colab-selenium[undetected]

In [200]:
%pip install selenium webdriver-manager

In [201]:
import time
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import google_colab_selenium as gs
from datetime import datetime, timedelta
import re

driver = gs.Chrome()

<IPython.core.display.Javascript object>

In [202]:
def get_published_date(relative_time):
    """Convert relative time (e.g., '2 hours ago') to actual date."""
    now = datetime.now()

    parts = relative_time.split()
    if len(parts) < 2:
        return now.strftime("%Y-%m-%d %H:%M")  # Return current time if format is unknown

    num, unit = int(parts[0]), parts[1]

    # Convert different time formats
    if "minute" in unit:
        return (now - timedelta(minutes=num)).strftime("%Y-%m-%d %H:%M")
    elif "hour" in unit:
        return (now - timedelta(hours=num)).strftime("%Y-%m-%d %H:%M")
    elif "day" in unit:
        return (now - timedelta(days=num)).strftime("%Y-%m-%d")
    elif "week" in unit:
        return (now - timedelta(weeks=num)).strftime("%Y-%m-%d")

    return now.strftime("%Y-%m-%d")  # Default case

In [203]:
def is_valid_date(date_text):
    """Check if the extracted text follows a valid date format."""
    date_pattern = r"^\d{1,2} [A-Za-z]+( \d{4})?$"  # Matches "11 Feb", "23 February", or "17 December 2024"
    return bool(re.match(date_pattern, date_text))

In [204]:
def bbc_articles():
    URL = 'https://www.bbc.com/news'
    response = requests.get(URL)

    if response.status_code == 200:
        print("The response code is:", response.status_code)

        soup = BeautifulSoup(response.content, 'html.parser')

        articles_data = []
        headlines = soup.find_all("h2", {"data-testid": "card-headline"})
        descriptions = soup.find_all("p", {"data-testid": "card-description"})
        published_dates = soup.find_all("span", {"data-testid": "card-metadata-lastupdated"})

        for i in range(len(headlines)):
            title = headlines[i].text.strip()
            description = descriptions[i].text.strip() if i < len(descriptions) else "No description"
            relative_time = published_dates[i].text.strip() if i < len(published_dates) else "No date"

            # Ensure relative_time is defined before using it
            if "ago" in relative_time:
                published_date = get_published_date(relative_time)
            else:
                published_date = relative_time  # Keep the original date if not relative

            articles_data.append({
                "published_date": published_date,
                "headline": title,
                "publisher": "BBC",
                "article_content": description,  # Short description as a placeholder for full content
                "category": "General"  # Needs to be mapped from URL or sections
            })

        print(f"BBC articles scraped: {len(articles_data)}")
        return articles_data
    else:
        print(f"Failed to retrieve BBC news. Status code: {response.status_code}")
        return []

# Run function
bbc_news = bbc_articles()

The response code is: 200
BBC articles scraped: 61


In [205]:
def bbc_sports_articles():
    URL = 'https://www.bbc.com/sport'
    response = requests.get(URL)

    if response.status_code == 200:
        print("The response code is:", response.status_code)

        soup = BeautifulSoup(response.content, 'html.parser')

        articles_data = []
        headlines = soup.find_all("p", class_="ssrcss-1b1mki6-PromoHeadline")  # Extract headlines
        descriptions = soup.find_all("p", {"data-testid": "card-description"})
        published_dates = soup.find_all("span", class_="ssrcss-1f39n02-VisuallyHidden")  # Extract actual date

        for i in range(len(headlines)):
            title = headlines[i].text.strip()
            description = descriptions[i].text.strip() if i < len(descriptions) else "No description"
            extracted_date = published_dates[i].text.strip() if i < len(published_dates) else "No date"

            # Process extracted date
            if "ago" in extracted_date:
                published_date = get_published_date(extracted_date)  # Convert relative time
            elif is_valid_date(extracted_date):
                try:
                    # Convert absolute date to YYYY-MM-DD if year is present
                    if len(extracted_date.split()) == 3:
                        published_date = datetime.strptime(extracted_date, "%d %B %Y").strftime("%Y-%m-%d")
                    else:
                        # Keep as-is for formats like "23 February"
                        published_date = extracted_date
                except ValueError:
                    published_date = extracted_date  # If conversion fails, store as-is
            else:
                print(f"Skipping invalid date: {extracted_date}")
                continue  # Skip if it's not a valid date

            articles_data.append({
                "published_date": published_date,
                "headline": title,
                "publisher": "BBC",
                "article_content": description,
                "category": "Sports"
            })

        print(f"BBC Sports articles scraped: {len(articles_data)}")
        return articles_data
    else:
        print(f"Failed to retrieve BBC Sports news. Status code: {response.status_code}")
        return []

# Run function
bbc_sports_news = bbc_sports_articles()

The response code is: 200
Skipping invalid date: BBC Homepage
Skipping invalid date: More menu
Skipping invalid date: More menu
Skipping invalid date: Close menu
Skipping invalid date: BBC Sport
Skipping invalid date: .
Skipping invalid date: .
Skipping invalid date: .
Skipping invalid date: Watch: YouTuber MrBeast crashes Formula E car. Video, 00:00:30
Skipping invalid date: Uncle Harry is 'the best supporter ever' - Lampard. Video, 00:02:01
Skipping invalid date: Mourinho falls asleep during reporter's long question. Video, 00:00:34
Skipping invalid date: Chelsea must finish in top four - Colwill. Video, 00:04:03
Skipping invalid date: 'Arriving in red will be cool' - Hamilton's Melbourne excitement. Video, 00:01:12
Skipping invalid date: .
Skipping invalid date: Russell won't 'bow down' to Verstappen. Video, 00:02:35
Skipping invalid date: Ferguson 'not carried away after brilliant result' Video, 00:00:58
Skipping invalid date: 'Liverpool must improve in PSG second leg' Video, 00:02

In [206]:
def bbc_business_articles():
    URL = 'https://www.bbc.com/news/business'
    response = requests.get(URL)

    if response.status_code == 200:
        print("The response code is:", response.status_code)

        soup = BeautifulSoup(response.content, 'html.parser')

        articles_data = []
        headlines = soup.find_all("h2", {"data-testid": "card-headline"})
        descriptions = soup.find_all("p", {"data-testid": "card-description"})
        published_dates = soup.find_all("span", {"data-testid": "card-metadata-lastupdated"})

        for i in range(len(headlines)):
            title = headlines[i].text.strip()
            description = descriptions[i].text.strip() if i < len(descriptions) else "No description"
            relative_time = published_dates[i].text.strip() if i < len(published_dates) else "No date"

            # Ensure relative_time is defined before using it
            if "ago" in relative_time:
                published_date = get_published_date(relative_time)
            else:
                published_date = relative_time  # Keep the original date if not relative

            articles_data.append({
                "published_date": published_date,
                "headline": title,
                "publisher": "BBC",
                "article_content": description,
                "category": "Business"
            })

        print(f"BBC Business articles scraped: {len(articles_data)}")
        return articles_data
    else:
        print(f"Failed to retrieve BBC Business news. Status code: {response.status_code}")
        return []

# Run function
bbc_business_news = bbc_business_articles()

The response code is: 200
BBC Business articles scraped: 42


In [207]:
def scrape_thestar_nation():
    URL = "https://www.thestar.com.my/news/nation"

    # Set up Selenium WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run in headless mode (optional)
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    # Use webdriver_manager to install ChromeDriver automatically
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(URL)
    wait = WebDriverWait(driver, 10)

    max_clicks = 10
    click_count = 0

    while click_count < max_clicks:
        try:
            # Scroll down to make the "Load More" button visible
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1.5)

            # Find and click the "Load More" button
            load_more_button = wait.until(EC.element_to_be_clickable((By.ID, "loadMorestories")))
            driver.execute_script("arguments[0].click();", load_more_button)

            click_count += 1
            print(f"Clicked Load More {click_count} time(s)...")

            # Wait for new articles to load
            time.sleep(2)

        except Exception:
            print("No more articles to load or Load More button not found.")
            break  # Exit loop when no more articles

    # Parse page source with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, "html.parser")

    articles_data = []
    news_items = soup.find_all("div", class_="row list-listing")  # Each news item

    for item in news_items:
        # Extract headline
        headline_tag = item.find("h2", class_="f18")
        headline = headline_tag.a.text.strip() if headline_tag and headline_tag.a else "No headline"

        # Extract description
        description_tag = item.find("p", style="overflow-wrap: break-word;")
        description = description_tag.text.strip() if description_tag else "No description"

        # Extract timestamp
        timestamp_tag = item.find("label", class_="timestamp")
        published_date = timestamp_tag.text.strip() if timestamp_tag else "No date"

        # Convert relative timestamps like "7m ago"
        if "ago" in published_date:
            published_date = datetime.now().strftime("%Y-%m-%d")

        articles_data.append({
            "published_date": published_date,
            "headline": headline,
            "publisher": "The Star",
            "article_content": description,
            "category": "Nation",
        })

    print(f"Total articles scraped: {len(articles_data)}")

    # Close the browser
    driver.quit()

    return articles_data

# Run scraper
thestar_nation_news = scrape_thestar_nation()

Clicked Load More 1 time(s)...
Clicked Load More 2 time(s)...
Clicked Load More 3 time(s)...
Clicked Load More 4 time(s)...
Clicked Load More 5 time(s)...
Clicked Load More 6 time(s)...
Clicked Load More 7 time(s)...
Clicked Load More 8 time(s)...
Clicked Load More 9 time(s)...
Clicked Load More 10 time(s)...
Total articles scraped: 110


In [208]:
    driver.quit()


In [209]:
def scrape_thestar_business():
    URL = "https://www.thestar.com.my/news/latest?tag=Business"

    # Set up Selenium WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run in headless mode (optional)
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(URL)
    wait = WebDriverWait(driver, 10)

    max_pages = 3  # Scrape page 1, then click page 2 and 3
    current_page = 1
    articles_data = []

    while current_page <= max_pages:
        print(f"Scraping page {current_page}...")

        # Parse page source with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, "html.parser")

        news_items = soup.find_all("li", class_="row")  # Each news item

        for item in news_items:
            # Extract headline
            headline_tag = item.find("h2", class_="f18")
            headline = headline_tag.a.text.strip() if headline_tag and headline_tag.a else "No headline"

            # Extract description
            description_tag = item.find("p", style="overflow-wrap: break-word;")
            description = description_tag.text.strip() if description_tag else "No description"

            # Extract timestamp
            timestamp_tag = item.find("time", class_="timestamp")
            published_date = timestamp_tag.text.strip() if timestamp_tag else "No date"

            # Convert relative timestamps like "7m ago"
            if "ago" in published_date:
                published_date = datetime.now().strftime("%Y-%m-%d")

            articles_data.append({
                "published_date": published_date,
                "headline": headline,
                "publisher": "The Star",
                "article_content": description,
                "category": "Business",
            })

        # Click on the next page number
        try:
            next_page_number = str(current_page + 1)  # Convert to string
            next_page_button = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, next_page_number)))
            next_page_button.click()
            time.sleep(3)  # Allow page to load
            current_page += 1
        except Exception as e:
            print(f"Could not find page {current_page + 1}: {e}")
            break  # Stop if there's no more pages to click

    print(f"Total articles scraped: {len(articles_data)}")

    # Close the browser
    driver.quit()

    return articles_data

# Run scraper
thestar_business_news = scrape_thestar_business()

Scraping page 1...
Scraping page 2...
Could not find page 3: HTTPConnectionPool(host='localhost', port=35889): Read timed out. (read timeout=120)
Total articles scraped: 40


In [ ]:
def scrape_nst_nation():
    URL = "https://www.nst.com.my/news/nation"

    # Set up Selenium WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run in headless mode (optional)
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(URL)
    wait = WebDriverWait(driver, 10)

    # Allow time for JS to load more articles
    time.sleep(10)

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Locate all news articles (corrected selector)
    news_items = soup.find_all("div", class_="col")  # Updated selector

    articles_data = []
    for item in news_items:
        # Extract headline
        headline_tag = item.find("h2", class_="node-title")
        headline = headline_tag.text.strip() if headline_tag else "No headline"

        # Extract description/snippet
        description_tag = item.find("div", class_="field-content")
        description = description_tag.text.strip() if description_tag else "No description"

        # Extract timestamp
        timestamp_tag = item.find("time", class_="created")
        published_date = timestamp_tag.text.strip() if timestamp_tag else "No date"

        # Convert relative timestamps like "7m ago"
        if "ago" in published_date:
            published_date = datetime.now().strftime("%Y-%m-%d")

        articles_data.append({
            "published_date": published_date,
            "headline": headline,
            "publisher": "NST",
            "article_content": description,
            "category": "Nation",
        })

    print(f"Total articles scraped: {len(articles_data)}")

    driver.quit()
    return articles_data

# Run scraper
nst_nation_news = scrape_nst_nation()

In [ ]:
# Convert to DataFrame
df_bbc = pd.DataFrame(bbc_news
                      + bbc_sports_news
                      + bbc_business_news
                      + thestar_nation_news
                      + thestar_business_news
                      + nst_nation_news)

# Save to CSV (optional)
df_bbc.to_csv("news.csv", index=False)

# Display DataFrame
print(df_bbc.shape)
df_bbc.head()